Firstly get all variables we are considering appended to a raw dataframe. 
Work on the 2visit first as its the largest.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast
import os

In [42]:



non_prog = pd.read_csv('Cohorts\\NonProgressors\\cleaned_20visit_nonprogressor.csv')
prog = pd.read_csv('Cohorts\\Progressors\\cleaned_20visit_progressors.csv')

# Read the CSV files and extract IDs
non_prog_ids = non_prog['NACCID'].tolist()
prog_ids = prog['NACCID'].tolist()
progvecs = prog.iloc[:, 1:].values
nonprogvecs = non_prog.iloc[:, 1:].values

# Create a new dataframe with IDs and labels
ids = non_prog_ids + prog_ids
labels = [0] * len(non_prog_ids) + [1] * len(prog_ids)
vectors = np.concatenate((nonprogvecs, progvecs), axis=0)

data = []
for id, label, vector in zip(ids, labels, vectors):
    for visit, value in enumerate(vector, start=1):
        data.append((id, visit, label, value))

df_combined = pd.DataFrame(data, columns=['ID', 'Visit', 'Prog_ID', 'Progression'])

# Set MultiIndex
#df_combined.set_index(['ID', 'Visit'], inplace=True)

# Print the DataFrame to verify
print(df_combined)

df_combined.to_csv('Cohorts\\20visit_combined.csv')

Empty DataFrame
Columns: [ID, Visit, Prog_ID, Progression]
Index: []


1-29-25
I am doing a check on some of these comborbidities to see if they just have one entry or have multiple, there is an issue with the add columns function converting everything to NaN so I'm going back and checking the dataset. 

In [17]:
# Create a dictionary to store the vectors for each ID
alcohol_vectors = {}
uds_df = pd.read_csv('uds.csv')
# Iterate through each unique ID in the uds_df
test_df = pd.read_csv('Cohorts\\10visit_combined.csv')
for id in test_df['ID'].unique():
    # Get the 'ALCOHOL' values for the current ID
    alcohol_values = uds_df.loc[uds_df['NACCID'] == id, 'ALCOHOL'].tolist()
    # Store the values in the dictionary
    alcohol_vectors[id] = alcohol_values

print (alcohol_vectors)
# # Print out only 5 vectors
# for i, (id, vector) in enumerate(alcohol_vectors.items()):
#     if i >= 5:
#         break
#     print(f"ID: {id}, Alcohol Vector: {vector}")


C:\Users\Andrew\AppData\Local\Temp\ipykernel_22884\3192254220.py:3: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  uds_df = pd.read_csv('uds.csv')


{'NACC031822': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC047473': [0, 0, 0, 0, 0, 0, 0, 0, 0, -4], 'NACC052337': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC235623': [0, 0, 0, 0, 0, -4, -4, -4, -4, -4], 'NACC253704': [2, 2, 2, 2, 2, -4, -4, -4, -4, -4], 'NACC299292': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC355742': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC437540': [0, 0, 2, 2, 2, 2, 2, -4, -4, -4], 'NACC482725': [0, 0, 0, 0, 0, -4, -4, -4, -4, -4], 'NACC654506': [0, 0, 0, 0, 0, -4, -4, -4, -4, -4], 'NACC697219': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC737174': [0, 0, 0, 0, 0, 0, -4, -4, -4, -4], 'NACC800986': [0, 0, 0, 0, 0, 0, 0, -4, -4, -4], 'NACC956352': [0, 0, 0, 0, -4, -4, -4, -4, -4, -4], 'NACC010592': [0, 0, -4, -4, -4, -4, -4, -4, -4, -4], 'NACC020061': [0, 0, 0, 0, 0, -4, -4, -4, -4, -4], 'NACC024469': [0, 0, -4, -4, -4, -4, -4, -4, -4, -4], 'NACC107477': [0, -4, -4, -4, -4, -4, -4, -4, -4, -4], 'NACC221139': [0, 0, 0, 0, 0, 0, 0, 0, 0, -4], 'NACC381154': [0, -4, -4, -4, -4, -4,

In [19]:
bmi_vectors = {}
uds_df = pd.read_csv('uds.csv')
# Iterate through each unique ID in the uds_df
test_df = pd.read_csv('Cohorts\\10visit_combined.csv')
for id in test_df['ID'].unique():
    # Get the 'BMI' values for the current ID
    bmi_values = uds_df.loc[uds_df['NACCID'] == id, 'NACCBMI'].tolist()
    # Store the values in the dictionary
    bmi_vectors[id] = bmi_values
print (bmi_vectors)

C:\Users\Andrew\AppData\Local\Temp\ipykernel_22884\3050750130.py:2: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  uds_df = pd.read_csv('uds.csv')


{'NACC031822': [27.1, 26.4, 27.8, 27.8, 26.4, 27.1, 888.8, 26.4, 29.0, 888.8], 'NACC047473': [27.1, 888.8, 21.0, 24.6, 25.6, 23.2, 23.7, 23.7, -4.0, 23.2], 'NACC052337': [26.4, 26.4, 27.3, 27.9, 30.0, -4.0, 28.7, 27.0, 888.8, 888.8], 'NACC235623': [27.5, 30.8, 28.7, 30.0, -4.0, 28.0, 25.9, 26.8, 888.8, -4.0], 'NACC253704': [28.0, 26.6, 27.0, 28.6, -4.0, 26.6, 28.4, 27.4, -4.0, 888.8], 'NACC299292': [23.0, 24.1, 22.1, 22.5, 25.4, -4.0, 24.1, 888.8, -4.0, -4.0], 'NACC355742': [27.1, 26.6, 26.2, 27.2, 27.8, -4.0, 27.2, 888.8, 27.7, 26.3], 'NACC437540': [26.6, 27.4, 28.8, 28.2, 27.1, -4.0, 28.5, 27.3, 26.7, 25.8], 'NACC482725': [32.3, 26.5, 26.2, 26.7, 25.6, 25.2, 26.0, -4.0, -4.0, 23.6], 'NACC654506': [25.5, 26.0, 26.3, 25.6, 26.5, -4.0, -4.0, -4.0, -4.0, -4.0], 'NACC697219': [25.9, 25.0, 25.5, 25.3, 25.0, -4.0, 888.8, 26.0, 26.2, 25.9], 'NACC737174': [32.1, 32.7, 33.2, 31.4, 34.1, -4.0, 32.9, 888.8, 888.8, 888.8], 'NACC800986': [23.8, 24.4, 25.8, 24.8, 24.7, 24.1, -4.0, 23.5, 22.7, 24.3]

In [20]:
mmse_vectors = {}
uds_df = pd.read_csv('uds.csv')
# Iterate through each unique ID in the uds_df
test_df = pd.read_csv('Cohorts\\10visit_combined.csv')
for id in test_df['ID'].unique():
    # Get the 'MMSE' values for the current ID
    mmse_values = uds_df.loc[uds_df['NACCID'] == id, 'NACCMMSE'].tolist()
    # Store the values in the dictionary
    mmse_vectors[id] = mmse_values
print (mmse_vectors)

C:\Users\Andrew\AppData\Local\Temp\ipykernel_22884\1335438444.py:2: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  uds_df = pd.read_csv('uds.csv')


{'NACC031822': [29, 29, 29, 28, 29, 27, -4, -4, -4, -4], 'NACC047473': [30, 30, 29, 29, 30, 27, 29, 30, -4, -4], 'NACC052337': [30, 30, 30, 30, 30, -4, -4, -4, -4, -4], 'NACC235623': [29, 30, 30, 30, -4, -4, -4, -4, -4, -4], 'NACC253704': [30, 29, 28, 30, -4, -4, -4, -4, -4, -4], 'NACC299292': [97, 97, 97, 97, 97, 97, -4, -4, -4, -4], 'NACC355742': [30, 29, 30, 29, 30, -4, -4, -4, -4, -4], 'NACC437540': [29, 30, 30, 30, 30, -4, 29, -4, -4, -4], 'NACC482725': [30, 30, 30, 30, 30, -4, -4, -4, -4, -4], 'NACC654506': [97, 97, 27, 29, 26, -4, -4, -4, -4, -4], 'NACC697219': [30, 29, 30, 30, 30, -4, -4, -4, -4, -4], 'NACC737174': [30, 30, 30, 30, 30, -4, -4, -4, -4, -4], 'NACC800986': [30, 30, 30, 30, 30, 30, -4, -4, -4, -4], 'NACC956352': [29, 30, 29, 29, -4, -4, -4, -4, -4, -4], 'NACC010592': [29, 28, -4, -4, -4, -4, -4, -4, -4, -4], 'NACC020061': [30, 29, 29, 30, 29, -4, -4, -4, -4, -4], 'NACC024469': [30, 29, -4, -4, -4, -4, -4, -4, -4, -4], 'NACC107477': [28, -4, -4, -4, -4, -4, -4, -4, 

In [ ]:
gds_vectors = {}
uds_df = pd.read_csv('uds.csv')
# Iterate through each unique ID in the uds_df
test_df = pd.read_csv('Cohorts\\10visit_combined.csv')
for id in test_df['ID'].unique():
    # Get the 'GDS' values for the current ID
    gds_values = uds_df.loc[uds_df['NACCID'] == id, 'NACCGDS'].tolist()
    # Store the values in the dictionary
    gds_vectors[id] = gds_values
print (gds_vectors)

In [8]:
def add_columns_from_uds(df_combined, uds_df):
    # Initialize the new columns with default values
    new_columns = ['ALCOHOL']

    # Iterate through the IDs in the first column of df_combined
    for id in df_combined['ID'].unique():
        for col in new_columns:
            values = uds_df.loc[uds_df['NACCID'] == id, col].tolist()
            if len(values) != len(df_combined[df_combined['ID'] == id]):
                values = [np.nan] * len(df_combined[df_combined['ID'] == id])
            df_combined.loc[df_combined['ID'] == id, col] = values
    
    # Drop the Visit column
    df_combined.drop(columns=['Visit'], inplace=True)
    
    return df_combined

# Load the global uds.csv DataFrame
uds_df = pd.read_csv('uds.csv', low_memory=False)
cohorts_dir = 'Cohorts'
# Iterate through every CSV in the Cohorts directory
for root, dirs, files in os.walk(cohorts_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df_combined = pd.read_csv(file_path)
            df_combined = add_columns_from_uds(df_combined, uds_df)
            
            # Print the updated DataFrame to verify
            print(f"Updated DataFrame for {file}:")
            print(df_combined.head())
            
            # Optionally, save the updated DataFrame back to CSV
            df_combined.to_csv(file_path, index=False)


Updated DataFrame for 10visit_combined.csv:
   Unnamed: 0          ID  Prog_ID                     Progression  SEX  EDUC  \
0           0  NACC031822        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    1    17   
1           1  NACC047473        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    17   
2           2  NACC052337        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    13   
3           3  NACC235623        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    12   
4           4  NACC253704        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    1    18   

  NACCFAM ALCOHOL NACCBMI HEARING  ... SHOPPING GAMES STOVE MEALPREP EVENTS  \
0     NaN     NaN     NaN     NaN  ...      NaN   NaN   NaN      NaN    NaN   
1     NaN     NaN     NaN     NaN  ...      NaN   NaN   NaN      NaN    NaN   
2     NaN     NaN     NaN     NaN  ...      NaN   NaN   NaN      NaN    NaN   
3     NaN     NaN     NaN     NaN  ...      NaN   NaN   NaN      NaN    NaN   
4     NaN     NaN     NaN     NaN  ...      NaN   NaN   NaN      NaN    Na

KeyboardInterrupt: 

In [18]:
columns_to_drop = ['Visit']

# Iterate through every CSV in the Cohorts directory
cohorts_dir = 'Cohorts'
for root, dirs, files in os.walk(cohorts_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
            
            # Print the updated DataFrame to verify
            print(f"Updated DataFrame for {file}:")
            print(df.head())
            
            # Optionally, save the updated DataFrame back to CSV
            df.to_csv(file_path, index=False)

Updated DataFrame for 10visit_combined.csv:
   Unnamed: 0          ID  Prog_ID                     Progression  SEX  EDUC
0           0  NACC031822        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    1    17
1           1  NACC047473        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    17
2           2  NACC052337        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    13
3           3  NACC235623        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2    12
4           4  NACC253704        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    1    18
Updated DataFrame for 11visit_combined.csv:
   Unnamed: 0          ID  Prog_ID                        Progression  SEX  \
0           0  NACC176170        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    1   
1           1  NACC181007        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2   
2           2  NACC245215        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2   
3           3  NACC291227        0  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)    2   
4           4  NACC421833        0  (0, 0, 0, 0, 0, 0,